## Base settings

In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
import time
import ua_generator
from ua_generator.options import Options as uaOptions
from ua_generator.data.version import VersionRange

from selenium.webdriver import Chrome


In [2]:
options = uaOptions()
options.weighted_versions = True
options.version_ranges = {
    'chrome': VersionRange(125, 129),  # Choose version between 125 and 129
    'edge': VersionRange(min_version=120),  # Choose version 120 minimum
}
ua = ua_generator.generate(platform=('linux', 'windows', 'windows'),
                           device='desktop', browser=('chrome', 'firefox'), options=options)
# Настройка Chrome
options = Options()
chrome_options = Options()
chrome_service = Service(executable_path="/usr/bin/chromedriver")

chrome_options.add_experimental_option(
    "excludeSwitches", ["enable-automation"])
chrome_options.add_experimental_option("useAutomationExtension", False)

# chrome_options.add_argument(f"user-agent={ua.text}")
chrome_options.add_argument('--disable-blink-features=AutomationControlled')
chrome_options.add_argument('--no-saда, ndbox')
chrome_options.add_argument('--disable-gpu')
chrome_options.add_argument("--enable-logging")
chrome_options.add_argument("--disable-blink-features")
chrome_options.add_argument("--disable-blink-features=AutomationControlled")


driver = Chrome(service=chrome_service, options=chrome_options)


## Emulate

In [3]:


"""
Object.defineProperty(navigator, 'userAgent', {
    value: 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36',
    configurable: true
});
"""


ultimate_script = """


// WebGLRenderingContext.getParameter - предотвращение рекурсии
const originalWebGLGetParameter = WebGLRenderingContext.prototype.getParameter;
WebGLRenderingContext.prototype.getParameter = function(parameter) {
    const params = {
        37446: "NVIDIA Corporation", // Подменим на данные, схожие с реальными
        37445: "NVIDIA GeForce RTX 3080/PCIe/SSE2",
        7936: "WebKit",
        7937: "WebGL 1.0"
    };
    return params[parameter] || Reflect.apply(originalWebGLGetParameter, this, arguments);
};

// Используем Object.defineProperty для подмены WebGL параметров
Object.defineProperty(WebGLRenderingContext.prototype, "RENDERER", {
    get: () => "NVIDIA GeForce RTX 3080"
});
Object.defineProperty(WebGLRenderingContext.prototype, "UNMASKED_VENDOR_WEBGL", {
    get: () => "NVIDIA Corporation"
});
Object.defineProperty(WebGLRenderingContext.prototype, "UNMASKED_RENDERER_WEBGL", {
    get: () => "NVIDIA GeForce RTX 3080/PCIe/SSE2"
});

// Навигационные свойства
Object.defineProperties(navigator, {
    languages: { get: () => ["en-US", "en"] },
    platform: { get: () => "Win32" },
    userAgent: {
        get: () => "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36"
    },
    vendor: { get: () => "Google Inc." },
    deviceMemory: { get: () => 8 },
    hardwareConcurrency: { get: () => 4 },
    maxTouchPoints: { get: () => 0 },
    mimeTypes: {
        get: () => ({
            0: { type: "application/pdf", suffixes: "pdf", description: "" },
            length: 1,
            item: function(index) {
                return this[index];
            }
        })
    },
    plugins: {
        get: () => ({
            0: { name: "Chrome PDF Viewer", description: "", filename: "internal-pdf-viewer" },
            length: 1,
            item: function(index) {
                return this[index];
            }
        })
    }
});

// Подменяем canvas
const originalToDataURL = HTMLCanvasElement.prototype.toDataURL;
HTMLCanvasElement.prototype.toDataURL = function() {
    return Reflect.apply(originalToDataURL, this, arguments) + "fake_data";
};

// Исправление рекурсии в DateTimeFormat
const originalResolvedOptions = Intl.DateTimeFormat.prototype.resolvedOptions;
Intl.DateTimeFormat.prototype.resolvedOptions = function() {
    const options = Reflect.apply(originalResolvedOptions, this, arguments);
    options.timeZone = "Asia/Yekaterinburg";
    return options;
};

// Подстановка appVersion
Object.defineProperty(navigator, "appVersion", {
    get: () => "5.0 (Windows NT 10.0; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/125.0.6422.103 Safari/537.36"
});

// Дополнительные параметры для Screen
Object.defineProperties(screen, {
    width: { get: () => 1920 },
    height: { get: () => 1080 },
    availWidth: { get: () => 1920 },
    availHeight: { get: () => 1040 },
    colorDepth: { get: () => 24 },
    pixelDepth: { get: () => 24 }
});
"""


In [4]:
driver.execute_cdp_cmd("Page.addScriptToEvaluateOnNewDocument", {
                       "source": ultimate_script})

# Открываем тестовую страницу
driver.get("https://abrahamjuliot.github.io/creepjs/")  # Тест на антибота